# Demo all Thunderbird WPS Processes

This notebook demonstrates how the scripts from PCIC's [climate-explorer-data-prep](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md) repository are used as Web Processing Service (WPS) processes in thunderbird. Before a process can be run, an instance of `thunderbird` must be activated by executing `thunderbird start` on a terminal. Each process takes netcdf files as input and, unless otherwise stated, outputs a metalink between the resulting netcdf files to allow them to be downloaded. Netcdf files from local storage or an opendap server can be used as input.

In [1]:
from birdy import WPSClient
import requests
from pkg_resources import resource_filename
from bs4 import BeautifulSoup

import os
os.chdir('../')

In [2]:
# Set up wps application
url = 'http://localhost:5001/wps'
thunderbird = WPSClient(url=url)

In [3]:
# Check wps info
thunderbird?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f03140e35f8>
File:            ~/thunderbird-venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Explorer data preparation

Processes
---------

generate_climos
    Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

generate_prsn
    Generate precipitation as snow file from precipitation and minimum/maximum temperature data

update_metadata
    Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

split_merged_climos
    Split climo means files into one file per time interval

hello
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.
Class docstring:
Returns a class where every public method is a WPS process available at
t

## Generate Climos

This process runs [generate_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_climos-generate-climatological-means), which creates files with climatological means/standard deviations of input data from a netcdf file. The `dry run` outputs metadata information about the input.

In [4]:
# Check info on `generate_climos` process
thunderbird.generate_climos?

Signature:
thunderbird.generate_climos(
    netcdf,
    operation,
    dry_run=None,
    convert_longitudes=True,
    split_vars=True,
    split_intervals=True,
    climo=None,
    resolutions=None,
)
Docstring:
Generate files containing climatological means from input files of daily, monthly, or yearly data that adhere to the PCIC metadata standard (and consequently to CMIP5 and CF standards).

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
operation : {'mean', 'std'}string
    Operation to perform on the datasets
climo : {'6190', '7100', '8110', '2020', '2050', '2080'}string
    Year ranges
resolutions : {'all', 'yearly', 'seasonal', 'monthly'}string
    Temporal Resolutions
convert_longitudes : boolean
    Transform longitude range from [0, 360) to [-180, 180)
split_vars : boolean
    Generate a separate file for each dependent variable in the file
split_intervals : boolean
    Generate a separate file f

In [5]:
# Dry run process
netcdf = 'http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc'
operation = 'mean'
climo = '6190'
resolutions = 'yearly'
dry_run = True

dry_output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=dry_run
)

# Process dry run output
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/fdd_seasonal_CanESM2_rcp85_r1i1p1_1951-2100.nc
climo_periods: {'6190'}
project: CMIP5
institution: PCIC
model: CanESM2
emissions: historical, rcp85
run: r1i1p1
dependent_varnames: ['fdd']
time_resolution: seasonal
is_multi_year_mean: False



In [6]:
# generate climos
output = thunderbird.generate_climos(
    netcdf=netcdf, 
    operation=operation, 
    climo=climo, 
    resolutions=resolutions, 
    dry_run=False
)

# Process normal output
req = requests.get(output.get()[0])
process = BeautifulSoup(req.content.decode('utf-8'))
print(process)

<?xml version="1.0" encoding="UTF-8"?><html><body><metalink xmlns="urn:ietf:params:xml:ns:metalink">
<published>2020-06-24T16:00:15Z</published>
<generator>PyWPS/4.2.2</generator>
<file name="fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc">
<identity>fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc</identity>
<size>191188</size>
<metaurl mediatype="application/x-netcdf">http://localhost:5001/outputs/77754738-b66e-11ea-afd0-c86000e3f2fd/fdd_aClimMean_BCCAQ_CanESM2_historical+rcp85_r1i1p1_19610101-19901231_Canada.nc</metaurl>
<publisher name="None" url="http://localhost:5001/wps"></publisher>
</file>
</metalink></body></html>


## Split Merged Climos

This process runs [split_merged_climos](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#split_merged_climos-split-climo-means-files-into-per-interval-files-month-season-year), which splits climatological means files into one file per time interval.

In [7]:
# Check info on `split_merged_climos` process
thunderbird.split_merged_climos?

Signature: thunderbird.split_merged_climos(netcdf=None, loglevel='INFO')
Docstring:
Split climo means files into one file per time interval

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF files to process
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/metalink+xml; version=4.0`
    Metalink object between output files
File:      ~/thunderbird/</home/slim/thunderbird-venv/lib/python3.6/site-packages/birdy/client/base.py-3>
Type:      method


In [8]:
# Test local and opendap files
tasmax_climos_local = resource_filename('tests', 'data/tiny_downscaled_tasmax_climos.nc')
hydromodel_climos_opendap = "http://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tiny_hydromodel_gcm_climos.nc"

output = thunderbird.split_merged_climos([tasmax_climos_local, hydromodel_climos_opendap])
req = requests.get(output.get()[0])
print(BeautifulSoup(req.content.decode('utf-8')))

<?xml version="1.0" encoding="UTF-8"?><html><body><metalink xmlns="urn:ietf:params:xml:ns:metalink">
<published>2020-06-24T16:00:18Z</published>
<generator>PyWPS/4.2.2</generator>
<file name="tasmax_mClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc">
<identity>/tmp/pywps_process_j2guymxb/tasmax_mClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc</identity>
<size>8164</size>
<metaurl mediatype="application/x-netcdf">http://localhost:5001/outputs/79066eba-b66e-11ea-afd0-c86000e3f2fd/tasmax_mClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc</metaurl>
<publisher name="None" url="http://localhost:5001/wps"></publisher>
</file>
<file name="tasmax_sClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc">
<identity>/tmp/pywps_process_j2guymxb/tasmax_sClim_BCCAQ2_ACCESS1-0_historical+rcp45_r1i1p1_19610101-19901231.nc</identity>
<size>8152</size>
<metaurl mediatype="application/x-netcdf">http://localhost:5001/outputs/79066ebb-b66e-11

## Update Metadata

This process runs [update_metadata](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#update_metadata-update-metadata-in-a-netcdf-file), which updates metadata from an input file according to instructions provided by a string or `.yaml` file. Rather than a metalink, the output is the updated netcdf file.

In [9]:
# Check info on `update_metadata` process
thunderbird.update_metadata?

Signature: thunderbird.update_metadata(netcdf, updates=None)
Docstring:
Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
updates : string
    The filepath of an updates file that specifies what to do to the metadata it finds in the NetCDF file

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf Files
File:      ~/thunderbird/</home/slim/thunderbird-venv/lib/python3.6/site-packages/birdy/client/base.py-2>
Type:      method


In [10]:
updates = resource_filename("tests", "metadata-conversion/simple-conversion.yaml")
netcdf = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
output = thunderbird.update_metadata(updates = updates, netcdf = netcdf)
print(output.get())

update_metadataResponse(
    output='http://localhost:5001/outputs/790e526a-b66e-11ea-afd0-c86000e3f2fd/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100_copy.nc'
)


## Generate Prsn

This process runs [generate_prsn](https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/README.md#generate_prsn-generate-snowfall-file), which generates a `snowfall_flux` file from precipitation, tasmin, and tasmax input files.

In [11]:
# Check info on `generate_prsn` process
thunderbird.generate_prsn?

Signature:
thunderbird.generate_prsn(
    prec,
    tasmin=None,
    tasmax=None,
    dry_run=None,
    chunk_size=100,
    output_file='None',
    loglevel='INFO',
)
Docstring:
Generate precipitation as snow file from precipitation and minimum/maximum temperature data

Parameters
----------
prec : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Precipitation file to process
tasmin : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmin file to process
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmax file to process
chunk_size : integer
    Number of time slices to be read/written at a time
output_file : string
    Optional custom name of output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
dry_run : boolean
    Checks file to ensure compatible with process

Returns
-------
output : ComplexData:mimetype:`applic

In [12]:
# Test local and opendap files
pr_file_local = resource_filename("tests", "data/pr_week_test.nc")
tasmin_file_opendap = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmax_file_opendap = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"

dry_output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True)
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: /tmp/pywps_process_8u50d9mi/pr_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['pr']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmin']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmax']



In [13]:
output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False, output_file="prsn_test_mixed.nc")
req = requests.get(output.get()[0])
print(BeautifulSoup(req.content.decode('utf-8')))

<?xml version="1.0" encoding="UTF-8"?><html><body><metalink xmlns="urn:ietf:params:xml:ns:metalink">
<published>2020-06-24T16:00:29Z</published>
<generator>PyWPS/4.2.2</generator>
<file name="prsn_test_mixed.nc">
<identity>prsn_test_mixed.nc</identity>
<size>7665520</size>
<metaurl mediatype="application/x-netcdf">http://localhost:5001/outputs/7f90f44e-b66e-11ea-afd0-c86000e3f2fd/prsn_test_mixed.nc</metaurl>
<publisher name="None" url="http://localhost:5001/wps"></publisher>
</file>
</metalink></body></html>
